In [2]:
import numpy as np
import pandas as pd
import math

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

import os

<h1>Loading Dataset</h1>

In [3]:
import os

category_class=sorted(['Auto','Bank','Blog','Business Interview','Economy','Education','Employment','Entertainment','Interview',
                'Literature','National News','Opinion','Sports','Technology','Tourism','World'])
df=pd.DataFrame(columns=['News','Category'])

#Temp array
temp=[]

#Convert texts file to dataframe
for i in range(0,len(category_class)):
    path=f"C://Users//Ghost//Desktop//training//Nepali//raw//{category_class[i]}"
    os.chdir(path)

    for file in os.listdir():
        # Check whether file is in text format or not
        if file.endswith(".txt"):
            file_path = f"{path}//{file}"
            
            #Exception Handling UnicodeDecode Error
            try:
                with open(file_path, 'r',encoding='utf-8') as f:
                    temp.append(f.read())
            except UnicodeDecodeError:
                pass
        
    
    df_copy=pd.DataFrame({"News": [x for x in temp],
                         "Category":f'{category_class[i]}'})
    df=pd.concat([df, df_copy], axis=0)
    temp=[]

In [4]:
df = df.sample(frac=1).reset_index(drop=True)

In [5]:
len(df)

14130

<h2>PreProcessing</h2>

In [54]:
df.groupby('Category').Category.count()

Category
Auto                    92
Bank                   344
Blog                   208
Business Interview     138
Economy               1176
Education               84
Employment             148
Entertainment         1164
Interview               86
Literature              16
National News         7329
Opinion                554
Sports                2268
Technology             103
Tourism                211
World                  209
Name: Category, dtype: int64

### Balancing

In [69]:
choice=['Sports','Opinion','National News','Entertainment','Economy']
index=[]
for i in range(len(df)):
    if df['Category'][i] in choice:
        drop_indices=np.random.choice(df.index,1,replace=False)
        index.append(drop_indices)
index

[array([12971], dtype=int64),
 array([12039], dtype=int64),
 array([12034], dtype=int64),
 array([1289], dtype=int64),
 array([463], dtype=int64),
 array([7011], dtype=int64),
 array([13745], dtype=int64),
 array([12707], dtype=int64),
 array([13718], dtype=int64),
 array([12066], dtype=int64),
 array([2763], dtype=int64),
 array([12418], dtype=int64),
 array([13303], dtype=int64),
 array([8502], dtype=int64),
 array([12790], dtype=int64),
 array([10433], dtype=int64),
 array([2623], dtype=int64),
 array([11231], dtype=int64),
 array([2733], dtype=int64),
 array([9604], dtype=int64),
 array([11542], dtype=int64),
 array([6563], dtype=int64),
 array([7545], dtype=int64),
 array([9875], dtype=int64),
 array([4899], dtype=int64),
 array([6716], dtype=int64),
 array([10097], dtype=int64),
 array([8758], dtype=int64),
 array([10003], dtype=int64),
 array([8085], dtype=int64),
 array([8900], dtype=int64),
 array([781], dtype=int64),
 array([8048], dtype=int64),
 array([6727], dtype=int64),
 

<h3>Maping</h3>

In [7]:
m={'Auto':0,
 'Bank':1,
 'Blog':2,
 'Business Interview':3,
 'Economy':4,
 'Education':5,
 'Employment':6,
 'Entertainment':7,
 'Interview':8,
 'Literature':9,
 'National News':10,
 'Opinion':11,
 'Sports':12,
 'Technology':13,
 'Tourism':14,
 'World':15}

In [8]:
df['Category_Id']=df['Category'].map(m)

<h3>Visualize</h3>

In [10]:
df["News"].sample(5)
print(df["News"][0])
df["News"][0].replace('\n','')


﻿१२ कात्तिक, जुम्ला । सडक पहुँच भएसँगै नेपालको सबैभन्दा ठूलो ताल रारा आन्तरिक तथा बाह्य पर्यटकको आकर्षणको थलो बन्दै गएको छ ।
रारा सबैको नजरमा पर्न त केही समय लाग्ला नै, तर यो वर्षको दशैंपछिको दुई साताभित्र रारा हेर्ने पर्यटकको सङ्ख्या करिब दुई हजार ६ सयको हाराहारीमा पुगेको छ । पर्यटन व्यवसायी गोपाल बमले भने – “असार, साउन र भदौसम्म पर्यटकको त्यति चहलपहल देखिएन, तर चैत, वैशाख, जेठ र असोजमा दैनिक २०–२५ आउने गरे पनि दशैंको दुई दिनपछि प्रतिदिन डेढ सयदेखि दुई सयको भीड लाग्न थाल्यो ।” अहिले यहाँका व्यवसायीलाई व्यवस्थापन गर्न हम्मेहम्मे परिरहेको छ ।
उनका  अनुसार यो समयमा धेरै आन्तरिक पर्यटकले रारा अवलोकन गरेका छन् । त्यसमा पनि ज्यादा जसो काठमाडौँ, भक्तपुर, ललितपुर, पोखरा, चितवन, भैरहवा, बुटवल, सुर्खेत र केही पूर्वी जिल्लाका पर्यटक रहेका छन् ।
बाह्य पर्यटक भने अमेरिका र युरोपेली मुलुकहरुबाट बढी मात्रामा आउने गरेका छन् । त्यहाँ पुग्ने पर्यटकलाई रारा राष्ट्रिय निकुञ्जद्वारा सञ्चालित हेरिटेज गेस्ट हाउस, पूर्णादेवी बमको डाँफे गेस्ट हाउस र तालबाट १५ मिनेट तल माझघट्टका होटलमा व्यवस्थापन गरिँदै आएको 

'\ufeff१२ कात्तिक,\xa0जुम्ला । सडक पहुँच भएसँगै नेपालको सबैभन्दा ठूलो ताल रारा आन्तरिक तथा बाह्य पर्यटकको आकर्षणको थलो बन्दै गएको छ ।रारा सबैको नजरमा पर्न त केही समय लाग्ला नै, तर यो वर्षको दशैंपछिको दुई साताभित्र रारा हेर्ने पर्यटकको सङ्ख्या करिब दुई हजार ६ सयको हाराहारीमा पुगेको छ । पर्यटन व्यवसायी गोपाल बमले भने – “असार, साउन र भदौसम्म पर्यटकको त्यति चहलपहल देखिएन, तर चैत, वैशाख, जेठ र असोजमा दैनिक २०–२५ आउने गरे पनि दशैंको दुई दिनपछि प्रतिदिन डेढ सयदेखि दुई सयको भीड लाग्न थाल्यो ।” अहिले यहाँका व्यवसायीलाई व्यवस्थापन गर्न हम्मेहम्मे परिरहेको छ ।उनका\xa0 अनुसार यो समयमा धेरै आन्तरिक पर्यटकले रारा अवलोकन गरेका छन् । त्यसमा पनि ज्यादा जसो काठमाडौँ, भक्तपुर, ललितपुर, पोखरा, चितवन, भैरहवा, बुटवल, सुर्खेत र केही पूर्वी जिल्लाका पर्यटक रहेका छन् ।बाह्य पर्यटक भने अमेरिका र युरोपेली मुलुकहरुबाट बढी मात्रामा आउने गरेका छन् । त्यहाँ पुग्ने पर्यटकलाई रारा राष्ट्रिय निकुञ्जद्वारा सञ्चालित हेरिटेज गेस्ट हाउस, पूर्णादेवी बमको डाँफे गेस्ट हाउस र तालबाट १५ मिनेट तल माझघट्टका होटलमा व्यवस्थापन गरिँ

In [11]:
stop_words_file=open("C:/Users/Ghost/Desktop/training/Nepali/stopwords.txt","r",encoding="utf-8")
stop_words=stop_words_file.read()
stop_words=stop_words.split("\n")

In [12]:
nepali_num_file=open("C:/Users/Ghost/Desktop/training/Nepali/numbers.txt","r",encoding="utf-8")
nepali_num=nepali_num_file.read()
nepali_num=nepali_num.split(",")

In [13]:
nepali_suffix_file=open("C:/Users/Ghost/Desktop/training/Nepali/suffix.txt","r",encoding="utf-8")
nepali_suffix=nepali_suffix_file.read()
nepali_suffix=nepali_suffix.split("\n")

<h3>PreProcessing Text</h3>

In [20]:
from Nepali_nlp import Tokenizer
def ProcessText(text):
    text=str(text)
    
    #removing \n and \ufeff
    remove=['\n','\ufeff']
    for i in remove:
        text.replace(i,'')
    
    #read stop words
    #Remove Stop Words
    word_tokens = Tokenizer().word_tokenize(text)
    filtered_list = [w for w in word_tokens if not w in stop_words]
    
    #Remove numbers
    num_filter=[]
    for i in range(0,len(filtered_list)):
        for j in range(0,len(nepali_num)):
            if nepali_num[j] in filtered_list[i]:
                num_filter.append(filtered_list[i])
                break
            
    for filter in num_filter:
        filtered_list.remove(filter)
    
    #Stemming Manual
    suffix_filter=filtered_list
    for i in range(len(nepali_suffix)):
        for j in range(len(suffix_filter)):
            if nepali_suffix[i] in suffix_filter[j]:
                suffix_removed=suffix_filter[j].split(nepali_suffix[i])[0]
                suffix_filter.remove(suffix_filter[j])
                suffix_filter.insert(j,suffix_removed)
    
    stemmed_string=' '.join(suffix_filter)
    
    return stemmed_string

In [21]:
print(ProcessText("\n एन्फाको २ ३अध्यक्ष पदमा पराजित उम्मेदवार कर्मा छिरिङ शेर्पाले साधारणसभामा आर्थिक चलखेल भएको आरोप लगाएका छन् । यद्यपी चुनावी परिणाम भने स्वीकार गर्ने उनले बताए ।"))

एन्फा पद पराजित उम्मेदवार कर् छिरिङ शेर्पा साधारणसभा आर्थिक चलखेल आरोप लगाए यद्यपी चुनावी परिणाम स्वी


<h3>Analyzing Processed Text</h3>

In [49]:
def rand():
    return np.random.binomial(n=1,p=0.01,size=[1])
def count_word(word):
    cnt=word.count(" ")+1
    return cnt
count=0
for i in range(0,len(df)):
    if rand():
        txt=df['News'][i]
        processed_txt=ProcessText(df['News'][i])
        
        print(txt+'\n \n'+processed_txt)
        print(f"\nMain Text Word Count : {count_word(txt)}\nProcessed Text Word Count : {count_word( processed_txt)}")
        count+=1
    if count==1:
        break
    


﻿राजविराज, ३ जेठ । सप्तरीको शम्भुनाथ नगरपालिका वडा नम्वर–७ कठौना बजार स्थित प्रहरी चौकी अगाडि खुल्लेआम ठगी भईरहँदा समेत प्रहरी मुकदर्शक बनेका छन् ।
जण्डिस रोग निको पारिदिने भन्दै सर्वसाधारणबाट पाँच हजारदेखि पच्चीस हजारसम्म एक नक्कली वैद्यले ठगीको धन्दा गरिरहँदा पनि प्रहरीले कार्वाही गर्न सकेको छैन । आफूलाई वैद्य बताउँदैआएका स्थानीय हरि प्रसाद पालले विगत एक दशकदेखि जण्डिस रोग निको पारिदिने भन्दै सर्वसाधारणलाई ठग्दैआएका छन् । प्रहरी चौकी कठौनाको मुख्य गेटमै उनले आफ्नै घरमा खुलेआम धन्दा गर्दै आएका छन् ।
जडीबुटीबाट जण्डिस रोग ग्यारेन्टिका साथ निको पारिदिने नाममा उनले दैनिकजसो चार–पाँच जनासँग रकम असुल्ने गरेका छन् । आफूलाई वैद्य बताउने पालले ईलाज गराउन आएका विरामीको आर्थिक हैसियत अनुसार रकम असुली गर्दैआएको पीडितहरुको बताउँछन् ।
पछिल्लो पटक धनुषाको गोदार–७ का दिलिप जयसवाल समेत नक्कली वैद्यको फन्दामा परे । जण्डिसबाट ग्रसित भएका उनी आज भन्दा करिब २०–२५ दिन अघि पालकहाँ ईलाज गराउन आएका थिए । पहिलो पटक उनले अन्ध्यारो कोठामा लगेर आफूलाई औषधी खुवाएको बताउँदै उनले भने ‘औषधी खुवाएपछि पाँच हजार रुपैय

<h3>Apply Process Text in our column </h3>

In [51]:
df['News']=df['News'].apply(ProcessText)

<h3>Transforming Text to Vectors for MultiClassification </h3>

In [63]:
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,ngram_range=(1,1))
features = tfidf.fit_transform(df['News']).toarray()
features.shape

(14130, 3733)

<h3>Displaying highly correlated words in categories </h3>

In [64]:
N = 5
category_id_df = df[['Category', 'Category_Id']].drop_duplicates()
category_id=dict(category_id_df.values)
labels=df['Category_Id']

for Category, category_id in sorted(category_id.items()):
  features_chi2 = chi2(features, labels == category_id)
  indices = np.argsort(features_chi2[0])
  feature_names = np.array(tfidf.get_feature_names_out())[indices]
  unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
  bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
  print(f"Category : {Category}")
  print(f"    Most Correlated Unigrams  :  {(', '.join(unigrams[-N:]))}")
  print(f"    Most Correlated Bigrams   :  {(', '.join(bigrams[-N:]))}")
  print("")

Category : Auto
    Most Correlated Unigrams  :  आइएमएस, ङय, अट, टरकन, एसय
    Most Correlated Bigrams   :  

Category : Bank
    Most Correlated Unigrams  :  बचत, तरलत, यरधन, भलपम, हकप
    Most Correlated Bigrams   :  

Category : Blog
    Most Correlated Unigrams  :  आएछ, आशल, रशमश, यसद, रमझम
    Most Correlated Bigrams   :  

Category : Business Interview
    Most Correlated Unigrams  :  यमश, तरर, ठअध, तसर, जखबर
    Most Correlated Bigrams   :  

Category : Economy
    Most Correlated Unigrams  :  णस, यवस, ऊर, दन, उपभ
    Most Correlated Bigrams   :  

Category : Education
    Most Correlated Unigrams  :  कक, एमव, फल, आइओएम, एसएलस
    Most Correlated Bigrams   :  

Category : Employment
    Most Correlated Unigrams  :  अरब, अदक, उसम, जग, दरख
    Most Correlated Bigrams   :  

Category : Entertainment
    Most Correlated Unigrams  :  चर, उड, नय, कथ, चलच
    Most Correlated Bigrams   :  

Category : Interview
    Most Correlated Unigrams  :  हरपल, झनल, खबरपत, तप, करश
    Most Correlat

In [60]:
df.head()

,News,Category,Category_Id
0,﻿ एसके शान सप्तरी मधेस आन्दोलन विरोध नारा गुञ...,National News,10
1,﻿आश्विन गत संविधान सभा निर्वाचन पार्टी उमेरद्...,National News,10
2,﻿ दिन ऊर्जा विषय थुप्रै बहस यस गत वैशाख भूकम्...,Blog,2
3,﻿ मंसिर नेपाल हुण्डाई गाडि आधि विक्रेता लक्ष्...,Auto,0
4,﻿उन भनाइ बेला महजोडी इच्छा राख्दा आचार्य ‘यस्त...,Entertainment,7


<h3>Train Test Split</h3>

In [62]:
x=features
y=df.loc[:,'Category_Id'].values
train_x,test_x,train_y,test_y=train_test_split(x,y,test_size=0.2,random_state=100)

<h2>Hyper Tuning</h2>

In [65]:
svc=SVC()
knn=KNeighborsClassifier()
dtc=DecisionTreeClassifier()
mb=MultinomialNB()
rtc=RandomForestClassifier()

In [67]:
models={'knn':knn,'dtc':dtc,'mb':mb,'rtc':rtc}

In [68]:
params={
    'knn': 
        {'n_neighbors':[3,5,7,9,11,13,15],
        'metric':['cosine','euclidean','manhattan'],
        'weights':['uniform','distance']},
    
    'svc': {'C':[0.1,1,10,100], 
            'gamma':[1,0.1,0.01,0.01], 
            'kernel':['rbf','linear']},
    
    'dtc':{
        'criterion':['gini','entropy'],
        'max_depth':[2,4,6,8,10,12]
    },
    
    'mb':{
        'alpha': [1.0,2.0],
    'fit_prior': [True]
    },
    'rtc':{
        'criterion':['gini','entropy'],
        'max_depth':[2,4,6,8,10,12]
    }
}

In [69]:
model_accuracy={}
score=0.0001
for model in models.keys():
    mod = GridSearchCV(
    models[model],
    params[model],
    verbose=0, #Progress bar showing
    cv=5, #cross validation
    n_jobs=-1, #cores to assign
)
    gridsearch_result=mod.fit(train_x,train_y)
    
    #selects best model
    if(score < float(gridsearch_result.score(test_x,test_y))):
            score=gridsearch_result.score(test_x,test_y)
            best_model=gridsearch_result
            
    predict=mod.predict(test_x)
    print(f"{model} : ",gridsearch_result.best_estimator_)
    if model not in model_accuracy.keys():
        model_accuracy.update({model:accuracy_score(test_y,predict)})

: 

In [26]:
model_accuracy

{'svc': 0.9798657718120806,
 'knn': 0.9563758389261745,
 'dtc': 0.7449664429530202,
 'mb': 0.9563758389261745,
 'rtc': 0.9261744966442953}

In [27]:
print(best_model.best_estimator_)
best_model.best_score_

SVC(C=10, gamma=1, kernel='linear')


0.9823775535318731

In [28]:
test_predict = best_model.predict(test_x)
train_accuracy = round(best_model.score(train_x,train_y)*100)
test_accuracy =round(accuracy_score(test_predict, test_y)*100)

print(f"Train Accuracy Score : {train_accuracy}")
print(f"Test Accuracy Score  : {test_accuracy}")
print()
print(classification_report(test_predict, test_y, target_names=Category_class))

Train Accuracy Score : 100
Test Accuracy Score  : 98

               precision    recall  f1-score   support

     business       1.00      0.98      0.99        65
entertainment       0.95      0.98      0.97        57
     politics       0.95      0.98      0.97        59
        sport       1.00      1.00      1.00        61
         tech       1.00      0.95      0.97        56

     accuracy                           0.98       298
    macro avg       0.98      0.98      0.98       298
 weighted avg       0.98      0.98      0.98       298



<h3>Predicting Text by Converting Text to Rough Tf-IDF</h3>


<h3>IDF Calculation</h3>

In [31]:

def calc_idf(Text_column):
    col=tfidf.get_feature_names_out()
    doc_count=len(Text_column)
    df={}
    idf={}
    for char in col:
        df[char]=0
        idf[char]=0
    #Calculating df
    for i in range(0,len(col)):
        for j in range(0,len(Text_column)):
            if col[i] in Text_column[j]:
                df[col[i]]+=1
    #Calculating idf
    for char in col:
        idf[char]=math.log((doc_count+1)/(1+df[char]))+1
    return(idf)

idf=calc_idf(df_train['Text'])


<h3>Above cell takes time to run so saving idf values as text For using Flask Operation</h3>

In [231]:
import ast
with open('backend/idf.txt', 'w') as f:
    print(idf, file=f)
with open('backend/idf.txt') as f:
     idf_load=f.read()
idf_load=ast.literal_eval(idf_load)

<h3>TF-IDF Calculation</h3>

In [32]:

def calc_tf_idf(txt,idf):
    idf=idf
    
    tf_idf={}
    word_count={}
    col=tfidf.get_feature_names_out()
    for ch in col:
        tf_idf[ch]=0
        word_count[ch]=0
    #Calculating tf
    words = txt.split()
    for ch in words:
        if ch in col:
            if ch in word_count:
                word_count[ch] += 1
            else:
                word_count[ch] = 1
   
    rough_tfidf=list(col)
    for keys in word_count.keys():
        tf_idf[keys]=idf[keys]*word_count[keys]
        if keys in rough_tfidf:
            index=rough_tfidf.index(keys)
            rough_tfidf[index]=tf_idf[keys]
    norm=0
    for i in range(0,len(rough_tfidf)):
        norm+=rough_tfidf[i]**2
    if norm==0:
        norm=1
    for i in range(0,len(rough_tfidf)):
        rough_tfidf[i]=round(rough_tfidf[i]/math.sqrt(norm),8)
    return rough_tfidf


In [33]:
#For Visualizing
arr=calc_tf_idf(" abandon abandon abandon  so abc abc  ability",idf)
print(tfidf.get_feature_names_out()[0:5])
print(arr[0:5])    

['abandon' 'abbott' 'abc' 'abilities' 'ability']
[0.79468979, 0.0, 0.58486913, 0.0, 0.16246917]


In [36]:
def Predict_text(txt):
    #process text
    text=ProcessText(txt)
    #calculate tfidf
    tf_idf=calc_tf_idf(txt,idf)
    #Validate if it is actually news
    if all(i > 0.05 for i in tf_idf) or count_word(txt)<100:
        print("Cannot classify")
    else:
        tf_idf_2=calc_tf_idf(" ",idf)
        tf_idf=np.append(tf_idf,tf_idf_2,axis=0)
        tf_idf=tf_idf.reshape(2,-1)
        #Predicting
        index=best_model.predict(tf_idf[0:1])
        print(Category_class[int(index)])

In [37]:
Predict_text("fox  too reliant on reality tv  the head of us tv network fox has admitted the broadcaster had relied too heavily on reality tv shows such as the poor-rating who s your daddy.  chief executive gail berman said  in the case of this fall we drifted to too much on the unscripted side . the series who s your daddy  where a young woman tries to pick her natural father for a cash prize caused outrage from adoption groups and rated badly. last season  fox s prime-time audience fell by 600 000 to 5.9 million. ms berman said:  i think the audience expects loud things from fox. sometimes they work  and sometimes they don t.   who s your daddy  the first episode of which was shown on 3 january  pulled in a disappointing audience of 6.3 million  according to the nielsen ratings system. five other episodes of the show had also been filmed will be dropped from fox s schedules  ms berman said. she was predicting a drop in ratings even for some of the network s established reality shows  such as american idol  which is due to start its fourth series this week. fox had unveiled a new strategy last year promising to launch new shows every season  including the traditionally quiet summer season. though that had met with a poor reception  ms berman said  there s no question that the audience  in our mind  is ready  willing and able to accept new programming in the summer . fox has changed this plan  launching new shows in may instead of june. one of the new shows will be the animated series american dad  made by seth macfarlane  the creator of family guy. that series  after becoming a hit on dvd  is also set to return with new episodes.")

entertainment


In [38]:
Predict_text("qpr keeper day heads for preston queens park rangers keeper chris day is set to join preston on a month s loan.  day has been displaced by the arrival of simon royce  who is in his second month on loan from charlton. qpr have also signed italian generoso rossi. r s manager ian holloway said:  some might say it s a risk as he can t be recalled during that month and simon royce can now be recalled by charlton.  but i have other irons in the fire. i have had a  yes  from a couple of others should i need them.   day s rangers contract expires in the summer. meanwhile  holloway is hoping to complete the signing of middlesbrough defender andy davies - either permanently or again on loan - before saturday s match at ipswich. davies impressed during a recent loan spell at loftus road. holloway is also chasing bristol city midfielder tom doherty.")

sport


In [214]:
Predict_text("eu referendum question unveiled the question to be asked in the referendum on the eu constitution has been unveiled by the government.  it will be:  should the united kingdom approve the treaty establishing a constitution for the european union   the constitution will be incorporated into uk law if there is a yes vote in the referendum  expected in 2006. critics say the constitution is a further step towards a federal europe  but advocates say it ensures effective operation of the enlarged 25-state eu.  if we reject this treaty  britain will be isolated and weak in europe   said foreign secretary jack straw  who along with the rest of the cabinet  will back a  yes  vote.  patriots by definition wanted the uk to be prosperous at home and strong and influential abroad  mr straw said.  our role as a leading member of the eu is a crucial part of securing that.   conservative shadow foreign secretary michael ancram said the referendum question  seems straightforward . but he accused the government of trying to confuse the issue by putting the eu referendum question in the same bill as the ratification of the constitution  when they should be treated as  two separate issues . despite this  underhand trick   the referendum bill stood  no chance of becoming law before the election   he added.  this is tony blair s cheap gesture to the pro-constitution lobby while he runs scared of a debate on europe he knows he cannot win.  neil o brien  director of anti-constitution group vote no  said:  the reality is that the government doesn t want to discuss the eu constitution ahead of the election because they know it is extremely unpopular with voters and with business.   the uk independence party said:  if the government believes that a no vote would mean that we should leave the european union  they should just ask us if we want to leave the eu. then we can be out of it and better off much sooner.   liberal democrat leader charles kennedy  who backs the constitution  said he expected the referendum would come in the first half of next year. he told bbc radio 4 s today programme:  the sooner we get on with this  the better.  he said the question sounded  very neutral  and  balanced   adding it would enable the argument  to be enjoined fairly and squarely on both sides . green mep caroline lucas welcomed tony blair s  courage in keeping his word  on holding a referendum. but she added:  this treaty is a flawed document that will make the eu less accountable  less sustainable  and less just.   mr blair signed the constitution at a ceremony in rome in november  but had already made it clear the issue would be put to voters in a referendum. that promise came after sustained pressure from opposition parties. jack straw  who argues the constitution reflected a  british vision for europe  and gives  national governments a stronger grip   has said the referendum could be held in spring 2006. but in an interview with the financial times  mr blair refused to be pinned down to that date  saying britain would hold a poll  some time in 2006 but when  i don t know . the paper said the prime minister  claimed ignorance  of when other countries were planning to hold their referendums.")

politics


In [215]:
Predict_text("french consumer spending rising french consumers increased their spending by 1.5% in january  a figure which bodes well for the country s economic growth  figures revealed.  the national statistic institute (insee) added that consumer spending in january rose 3.8% on a year-on-year basis. rising sales of household equipment were behind the increase. the insee also said that french consumer prices fell 0.6% in january  but were up 1.6% on an annual basis.  despite the general increase in spending in january  french households bought fewer cars in january. according to the insee  car sales fell 2.8% in january  following a fall of 0.6% in december. but on a year-on-year basis  the sector still saw a sales increase of 6.5%. consumer spending fuelled france s economic growth in the last quarter of 2004 and analysts expect that it will continue to support the economy.  it s a growth that will remain fragile and vulnerable to risks like a strong rise in long-term interest rates  tension in the oil price   emmanuel ferry  from exane bnp paribas told reuters news agency.  meanwhile in italy  consumer confidence rose to its highest level since october 2004. economic research group isae has said that italian consumer confidence rose to 104.4 from 103.3  despite a slight deterioration in short-term sentiment.")

business


In [216]:
Predict_text("search sites get closer to users search sites want to get to know you better.  not content with providing access to the millions of websites  many now offer ways that do a better job of remembering  cataloguing and managing all the information you come across. some of the latest to update their search systems are ask jeeves and blinkx  which have both released a series of utilities that try to help people get more from the web.  the future is all about developing your own personal web   said tony macklin  spokesman for ask jeeves.  mr macklin said that too often when people use a search engine it was like the first time they ever used it  because there was no memory of what they had searched for before.  each time you go back in you have to start all over again   he said. the series of updates to its service  collected under the my ask jeeves banner  would help people remember where they had been before.  ask jeeves has added the ability to  save  websites of interest so the next time a users visits the site they can search through the sites they have previously found. sites saved in this way can be arranged in folders and have notes attached to them to explain why they were saved. mr macklin said many people wanted to save sites they had seen but did not want to add them to their bookmarks or favourites not least because such lists cannot be easily searched. on average  said mr macklin  users conduct between five and 10 searches per day and the tools in my ask jeeves should stop them having to do searches twice and get to what they want much more easily. under my ask jeeves users can search the web or through the results they have already noted as interesting.  it s about finding again what you found before   he said. the my ask jeeves service lets people store up to a 1000 web links or 5000 if they sign up to the free service. by way of comparison google s desktop search tool catalogues search histories informally and lets people look through the sites they have visited.  at the same time  search start-up blinkx has released a second version of its eponymous software. blinkx is desktop search software that watches what someone is working on  be it a document or e-mail  and suggests websites  video clips  blogs or documents on a pc that are relevant to it.  since blinkx launched it has faced increased competition from firms such as google  copernic  enfish  x1 and apple all of whom now have programs that let people search their pc as well as the web.  the competition has validated the problem we tackle   said suranga chandratillake  co-founder of blinkx. in the latest release of blinkx  the company has added what it calls smart folders. once created the folders act as persistent queries that automatically sweep the web for pages related to their subject and catalogues relevant information  documents or incoming e-mails  on hard drives too. what users do with blinkx and other desktop search engines shows that people tend to be very promiscuous in their use of search engines.  blinkx users do not stop using other web search systems   he said.  they might use google to look up a company  or yahoo for travel because they know they are good at that   he said.  the classic thing we have seen recently  is people using blinkx to look at the things they have searched on   he said. the variety of ways to search data was only helping users  said mr chandratillake and that it was likely that in the future people would use different ones for different tasks.")

tech


In [232]:
import joblib

In [233]:
joblib.dump(best_model,'model.pkl')

['model.pkl']

In [33]:
model=joblib.load('model.pkl')

In [191]:
def Predict_text_pickle(txt):
    #process text
    text=ProcessText(txt)
    #calculate tfidf
    tf_idf=calc_tf_idf(txt,idf)
    #Validate if it is actually news
    if all(i <.05 for i in tf_idf) or count_word(txt)<100:
        print("Cannot classify")
    else:
        tf_idf_2=calc_tf_idf(" ",idf)
        tf_idf=np.append(tf_idf,tf_idf_2,axis=0)
        tf_idf=tf_idf.reshape(2,-1)
        #Predicting
        index=model.predict(tf_idf[0:1])
        print(Category_class[int(index)])

In [189]:
Predict_text_pickle("The nepali film industry will need some lifting.")
Predict_text_pickle(" ")

Cannot classify
Cannot classify


In [192]:
Predict_text_pickle("fox  too reliant on reality tv  the head of us tv network fox has admitted the broadcaster had relied too heavily on reality tv shows such as the poor-rating who s your daddy.  chief executive gail berman said  in the case of this fall we drifted to too much on the unscripted side . the series who s your daddy  where a young woman tries to pick her natural father for a cash prize caused outrage from adoption groups and rated badly. last season  fox s prime-time audience fell by 600 000 to 5.9 million. ms berman said:  i think the audience expects loud things from fox. sometimes they work  and sometimes they don t.   who s your daddy  the first episode of which was shown on 3 january  pulled in a disappointing audience of 6.3 million  according to the nielsen ratings system. five other episodes of the show had also been filmed will be dropped from fox s schedules  ms berman said. she was predicting a drop in ratings even for some of the network s established reality shows  such as american idol  which is due to start its fourth series this week. fox had unveiled a new strategy last year promising to launch new shows every season  including the traditionally quiet summer season. though that had met with a poor reception  ms berman said  there s no question that the audience  in our mind  is ready  willing and able to accept new programming in the summer . fox has changed this plan  launching new shows in may instead of june. one of the new shows will be the animated series american dad  made by seth macfarlane  the creator of family guy. that series  after becoming a hit on dvd  is also set to return with new episodes.")

entertainment


In [193]:
Predict_text_pickle("qpr keeper day heads for preston queens park rangers keeper chris day is set to join preston on a month s loan.  day has been displaced by the arrival of simon royce  who is in his second month on loan from charlton. qpr have also signed italian generoso rossi. r s manager ian holloway said:  some might say it s a risk as he can t be recalled during that month and simon royce can now be recalled by charlton.  but i have other irons in the fire. i have had a  yes  from a couple of others should i need them.   day s rangers contract expires in the summer. meanwhile  holloway is hoping to complete the signing of middlesbrough defender andy davies - either permanently or again on loan - before saturday s match at ipswich. davies impressed during a recent loan spell at loftus road. holloway is also chasing bristol city midfielder tom doherty.")

sport


In [194]:
Predict_text_pickle("eu referendum question unveiled the question to be asked in the referendum on the eu constitution has been unveiled by the government.  it will be:  should the united kingdom approve the treaty establishing a constitution for the european union   the constitution will be incorporated into uk law if there is a yes vote in the referendum  expected in 2006. critics say the constitution is a further step towards a federal europe  but advocates say it ensures effective operation of the enlarged 25-state eu.  if we reject this treaty  britain will be isolated and weak in europe   said foreign secretary jack straw  who along with the rest of the cabinet  will back a  yes  vote.  patriots by definition wanted the uk to be prosperous at home and strong and influential abroad  mr straw said.  our role as a leading member of the eu is a crucial part of securing that.   conservative shadow foreign secretary michael ancram said the referendum question  seems straightforward . but he accused the government of trying to confuse the issue by putting the eu referendum question in the same bill as the ratification of the constitution  when they should be treated as  two separate issues . despite this  underhand trick   the referendum bill stood  no chance of becoming law before the election   he added.  this is tony blair s cheap gesture to the pro-constitution lobby while he runs scared of a debate on europe he knows he cannot win.  neil o brien  director of anti-constitution group vote no  said:  the reality is that the government doesn t want to discuss the eu constitution ahead of the election because they know it is extremely unpopular with voters and with business.   the uk independence party said:  if the government believes that a no vote would mean that we should leave the european union  they should just ask us if we want to leave the eu. then we can be out of it and better off much sooner.   liberal democrat leader charles kennedy  who backs the constitution  said he expected the referendum would come in the first half of next year. he told bbc radio 4 s today programme:  the sooner we get on with this  the better.  he said the question sounded  very neutral  and  balanced   adding it would enable the argument  to be enjoined fairly and squarely on both sides . green mep caroline lucas welcomed tony blair s  courage in keeping his word  on holding a referendum. but she added:  this treaty is a flawed document that will make the eu less accountable  less sustainable  and less just.   mr blair signed the constitution at a ceremony in rome in november  but had already made it clear the issue would be put to voters in a referendum. that promise came after sustained pressure from opposition parties. jack straw  who argues the constitution reflected a  british vision for europe  and gives  national governments a stronger grip   has said the referendum could be held in spring 2006. but in an interview with the financial times  mr blair refused to be pinned down to that date  saying britain would hold a poll  some time in 2006 but when  i don t know . the paper said the prime minister  claimed ignorance  of when other countries were planning to hold their referendums.")

politics


In [195]:
Predict_text_pickle("software watching while you work software that can not only monitor every keystroke and action performed at a pc but also be used as legally binding evidence of wrong-doing has been unveiled.  worries about cyber-crime and sabotage have prompted many employers to consider monitoring employees. the developers behind the system claim it is a break-through in the way data is monitored and stored. but privacy advocates are concerned by the invasive nature of such software.  the system is a joint venture between security firm 3ami and storage specialists bridgehead software. they have joined forces to create a system which can monitor computer activity  store it and retrieve disputed files within minutes. more and more firms are finding themselves in deep water as a result of data misuse. sabotage and data theft are most commonly committed from within an organisation according to the national hi-tech crime unit (nhtcu) a survey conducted on its behalf by nop found evidence that more than 80% of medium and large companies have been victims of some form of cyber-crime. bridgehead software has come up with techniques to prove  to a legal standard  that any stored file on a pc has not been tampered with. ironically the impetus for developing the system came as a result of the freedom of information act  which requires companies to store all data for a certain amount of time.  the storage system has been incorporated into an application developed by security firm 3ami which allows every action on a computer to be logged. potentially it could help employers to follow the trail of stolen files and pinpoint whether they had been emailed to a third party  copied  printed  deleted or saved to cd  floppy disk  memory stick or flash card. other activities the system can monitor include the downloading of pornography  the use of racist or bullying language or the copying of applications for personal use. increasingly organisations that handle sensitive data  such as governments  are using biometric log-ins such as fingerprinting to provide conclusive proof of who was using a particular machine at any given time. privacy advocates are concerned that monitoring at work is not only damaging to employee s privacy but also to the relationship between employers and their staff.  that is not the case   said tim ellsmore  managing director of 3ami.  it is not about replacing dialogue but there are issues that you can talk through but you still need proof   he said.  people need to recognise that you are using a pc as a representative of a company and that employers have a legal requirement to store data   he added.")

tech


In [197]:
Predict_text_pickle("french consumer spending rising french consumers increased their spending by 1.5% in january  a figure which bodes well for the country s economic growth  figures revealed.  the national statistic institute (insee) added that consumer spending in january rose 3.8% on a year-on-year basis. rising sales of household equipment were behind the increase. the insee also said that french consumer prices fell 0.6% in january  but were up 1.6% on an annual basis.  despite the general increase in spending in january  french households bought fewer cars in january. according to the insee  car sales fell 2.8% in january  following a fall of 0.6% in december. but on a year-on-year basis  the sector still saw a sales increase of 6.5%. consumer spending fuelled france s economic growth in the last quarter of 2004 and analysts expect that it will continue to support the economy.  it s a growth that will remain fragile and vulnerable to risks like a strong rise in long-term interest rates  tension in the oil price   emmanuel ferry  from exane bnp paribas told reuters news agency.  meanwhile in italy  consumer confidence rose to its highest level since october 2004. economic research group isae has said that italian consumer confidence rose to 104.4 from 103.3  despite a slight deterioration in short-term sentiment.")

business
